In [2]:
#Import dependencies

from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
from pprint import pprint
from urllib.request import urlopen, Request
from datetime import datetime
from config import password

# Abbr

In [3]:
#Decided to find a website to scrape the state abbreviations
url_abv = "https://abbreviations.yourdictionary.com/articles/state-abbrev.html"

In [4]:
response = requests.get(url_abv)
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
#HTML inspect found the values in the li tag
results = soup.find_all('li')

In [6]:
# total length of li
len(results)

250

In [7]:
#strip out only the first 50 li tags that contain the abbreviations
ABV = []

In [8]:
for i in range(0, 50):
    results[i].text
    ABV.append(results[i].text)

In [9]:
#Take the new lists and seperate the abbreviations from state names into a new lists
ABV_new = []
State_name = []

In [10]:
#abbreviation list
for i in range(0, 50):
    ABV_clean = ABV[i].split(' - ')[1]
    ABV_new.append(ABV_clean.replace(' ',''))

In [11]:
# #state name list
# for i in range(0, 50):
#     ABV_clean = ABV[i].split(' - ')[0]
#     State_name.append(ABV_clean.replace(' ',''))

In [12]:
#create dataframe
All_States_df = pd.DataFrame({'Abbreviation':ABV_new})

In [13]:
#Get chromedriver and splinter  up and running

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [14]:
#set url for the article, paragraph, and image

url="https://www.bls.gov/charts/state-employment-and-unemployment/state-unemployment-rates-animated.htm"
browser.visit(url)

In [15]:
browser.click_link_by_partial_text('Show table')

In [16]:
RawData = browser.html

In [17]:
tables = pd.read_html(RawData)

In [18]:
df = tables[0]
clean_df = df.drop([8,51])
#set index to state to make sure all columns are dates
clean_df = clean_df.set_index("State")

#convert all column lables to datetime and format as month-year
clean_df.columns = pd.to_datetime(clean_df.columns).strftime("%b-%Y")

#reset index to get state as it's own column
clean_df = clean_df.reset_index()


In [19]:
result = pd.concat([clean_df, All_States_df], axis=1, sort=False)

In [20]:
result[result["Abbreviation"]=="NY"]

,State,Feb-2009,Mar-2009,Apr-2009,May-2009,Jun-2009,Jul-2009,Aug-2009,Sep-2009,Oct-2009,...,Jun-2018,Jul-2018,Aug-2018,Sep-2018,Oct-2018,Nov-2018,Dec-2018,Jan-2019,Feb-2019,Abbreviation
31,New York,7.5,7.8,8.1,8.3,8.4,8.5,8.7,8.8,8.8,...,4.1,4.0,3.9,3.8,3.8,3.9,3.9,3.9,3.9,NY


In [21]:
import pymysql
from config import password
pymysql.install_as_MySQLdb()

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float

from sqlalchemy import create_engine

In [22]:
dbuser = 'root'
dbpassword = password
dbhost = 'localhost'
dbport = '3306'
dbname= 'project2'

engine = create_engine(f"mysql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}")

result.to_sql('unemployment', engine, index=True, if_exists="replace")

In [25]:
result[result["Abbreviation"]=="NY"].T

,31
State,New York
Feb-2009,7.5
Mar-2009,7.8
Apr-2009,8.1
May-2009,8.3
Jun-2009,8.4
Jul-2009,8.5
Aug-2009,8.7
Sep-2009,8.8
Oct-2009,8.8
